In [2]:
import pandas as pd
import yaml
from pathlib import Path


In [3]:
data_dir = Path('../data-dir/inputs/manual').resolve()
region_countries = ['US', 'CA', 'AU', 'CN']
gleam_reg_df = pd.read_csv(data_dir / 'regions-gleam.csv')
reg_df = pd.read_csv(data_dir / 'regions.csv')
reg_agg = {}
# with (data_dir / 'regions-agg.yaml').open('r') as f:
#     reg_agg = yaml.load(f)

aggregated_regions = []

In [ ]:
for region in region_countries:
    sub_regions = reg_df\
        .loc[reg_df['Code'].str.startswith(f'{region}-', na=False), ['Code', 'Name', 'Population']]\
        .set_index('Code')\
        .to_dict(orient='index')
    for code, agg in sub_regions.items():
        gleam_regions = list(gleam_reg_df.loc[gleam_reg_df['SubdivisionCode'] == code, 'Code'])
        aggregated_regions += gleam_regions
        print(gleam_regions)
        if agg['AggregateFrom']:
            agg['AggregateFrom'] = gleam_regions
            reg_agg[code] = agg
        else:
            print(f"No subdivisions for {code}")

In [9]:
gleam_reg_df.loc[gleam_reg_df.Code.isin(aggregated_regions)]

,Code,Name,OfficialName,OtherNames,Level,M49Code,ContinentCode,SubregionCode,CountryCode,CountryCodeISOa3,SubdivisionCode,Lat,Lon,Population,GleamID


In [78]:
with (data_dir / 'regions-agg.yaml').open('a') as f:
    reg_agg = yaml.dump(reg_agg, f)

In [15]:
with (data_dir / 'regions-agg.yaml').open('r') as f:
    reg_agg = yaml.load(f)

aggregated_regions = sum([
    list(value['AggregateFrom'].keys())
    if isinstance(value['AggregateFrom'], dict)
    else value['AggregateFrom']
    for value in reg_agg.values()
], [])

/home/radyop/bs/covid/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [18]:
gleam_reg_df.loc[gleam_reg_df.Code.isin(aggregated_regions), 'SubdivisionCode'] = pd.np.nan


/home/radyop/bs/covid/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [21]:
gleam_reg_df.set_index('Code').to_csv(data_dir / 'regions-gleam_cleaned.csv')
